In [106]:
import numpy as np
import torch 

class SimpleBandit:
    def __init__(self, n_levers):
        self.n_levers = n_levers
        self.rewards = np.random.randint(20, size=n_levers)

    def context(self):
        return np.random.randint(self.n_levers) / self.n_levers

    def pull(self, lever, context):
        good_target = round(context.item() * self.n_levers)
        if lever == good_target:
            return 30

        return self.rewards[lever]

     

In [107]:
round(contexts[40].item() * 10)

4

In [108]:
class AgentNN(torch.nn.Module):
    def __init__(self, n_levers):
        super().__init__()
        self._model = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, n_levers)
        )
    
    def forward(x):
        return self._model(x)

class CriticNN(torch.nn.Module):
    def __init_(self):
        super().__init__()
        self._model = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 1)
        )
    
    def forward(x):
        return self._model(x)

In [109]:
bandit = SimpleBandit(10)
actor = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, bandit.n_levers),
            torch.nn.Softmax(dim=-1)
        )
critic = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 1)
        )

In [110]:
actor_opt = torch.optim.Adam(actor.parameters(), lr=1e-3)
critic_opt = torch.optim.Adam(critic.parameters(), lr=1e-3)
mse = torch.nn.MSELoss()

In [111]:
batch_size = 1024
contexts = torch.zeros(1024, 1)

for i in range(100000):
    contexts = torch.tensor([bandit.context() for _ in actions], dtype=torch.float).reshape(-1, 1)

    probs = actor(contexts)
    probs_flat = torch.full(probs.shape, 1/bandit.n_levers)
    actions = torch.multinomial(probs, num_samples=1).squeeze()
    explore = torch.bernoulli(torch.full(probs.shape[:1], 0.05))
    actions = (1 - explore) * actions + explore * torch.multinomial(probs_flat, num_samples=1).squeeze()
    actions = actions.long()

    values = critic(contexts)
    rewards = torch.tensor([bandit.pull(a, c) for a, c in zip(actions, contexts)], dtype=torch.float)

    print(f'{i}: {rewards.mean()}')

    action_probs = probs.gather(1, actions.view(-1, 1))
    advantage = rewards - values

    actor_loss = -(torch.log(action_probs) * advantage).mean()
    critic_loss = mse(values, rewards)

    actor_opt.zero_grad()
    actor_loss.backward(retain_graph=True)
    actor_opt.step()
    
    critic_opt.zero_grad()
    critic_loss.backward()
    critic_opt.step()


0: 10.96875
1: 10.33984375
2: 10.3017578125
3: 10.1552734375
4: 10.259765625
5: 10.314453125
6: 10.1142578125
7: 9.900390625
8: 10.326171875
9: 10.765625
10: 10.44921875
11: 11.0517578125
12: 10.3701171875
13: 9.8857421875
14: 10.3623046875
15: 10.6767578125
16: 10.4677734375
17: 10.5859375
18: 9.8642578125
19: 10.716796875
20: 10.6396484375
21: 10.6201171875
22: 10.265625
23: 10.56640625
24: 10.4765625
25: 10.7890625
26: 10.3994140625
27: 10.275390625
28: 10.2041015625
29: 10.1533203125
30: 10.3466796875
31: 10.5556640625
32: 10.9111328125
33: 10.794921875
34: 10.66015625
35: 10.4443359375
36: 10.5517578125
37: 11.05078125
38: 10.5234375
39: 10.19140625
40: 10.458984375
41: 10.70703125
42: 10.4072265625
43: 10.2041015625
44: 10.3271484375
45: 10.47265625
46: 10.36328125
47: 10.42578125
48: 9.98828125
49: 10.771484375
50: 10.4638671875
51: 10.0068359375
52: 10.419921875
53: 10.5400390625
54: 10.75
55: 10.0732421875
56: 10.0283203125
57: 10.4208984375
58: 10.396484375
59: 10.279296875
6

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [98]:
probs

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<SoftmaxBackward0>)

In [92]:
torch.multinomial(probs_flat, num_samples=1).squeeze().shape

torch.Size([1024])

In [89]:
explore * torch.multinomial(probs_flat, num_samples=1).squeeze()

RuntimeError: The size of tensor a (10) must match the size of tensor b (1024) at non-singleton dimension 1

In [42]:
actions

tensor([2, 4, 2,  ..., 7, 1, 2])

In [44]:
action_probs

tensor([[0.1081],
        [0.0899],
        [0.1081],
        ...,
        [0.1022],
        [0.0912],
        [0.1081]], grad_fn=<GatherBackward0>)

In [26]:
log_probs = actor(contexts)

In [30]:
probs = actor(contexts)
actions = torch.multinomial(probs, num_samples=1).squeeze()

In [31]:
actions

tensor([4, 9, 3,  ..., 8, 1, 7])

In [23]:
probs

tensor([[0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145],
        [0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145],
        [0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145],
        ...,
        [0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145],
        [0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145],
        [0.8347, 0.9481, 1.1612,  ..., 1.0587, 0.9021, 0.8145]],
       grad_fn=<ExpBackward0>)